## Timetabling Program for St. Margaret's School (Multiple Trials with Graph Colouring)

In [26]:
#import sys
#!{sys.executable} -m pip install ortools 

# import modules
import time
import numpy as np
import pandas as pd
import networkx as nx
import grinpy as gp
from random import random
from ortools.linear_solver import pywraplp

In [27]:
# Import Student Data
StudentInput = pd.ExcelFile("SMS Student Data.xlsx") 
StudentMatrix = pd.read_excel(StudentInput, 'Data') 
StudentInfo = StudentMatrix.values.tolist()
StudentMatrix[1:5]

,NAME,Grade,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
1,S1,11,0,0,29,31,34,45,61,66,74,68
2,S2,11,16,0,29,34,47,55,61,71,74,0
3,S3,11,0,0,29,34,47,61,0,74,0,0
4,S4,11,0,29,34,46,52,63,68,75,0,45


In [28]:
print('58 students requested a total of', np.count_nonzero(StudentMatrix.iloc[1:, 2:]), 'courses')
print('which is fewer than the max total of', 58*9)

58 students requested a total of 447 courses
which is fewer than the max total of 522


In [29]:
# Import Course Data
CourseInput = pd.ExcelFile("SMS Course Data.xlsx") 
CourseMatrix = pd.read_excel(CourseInput, 'Data') 
CourseInfo = CourseMatrix.values.tolist()
CourseMatrix[1:5]

,Course Name,CourseID,Section,ABC,Teacher,FixedBlock,Core
1,Arts Education / ADST [Art Studio 11],3,1,NO,Louise,0,NO
2,Arts Education / ADST [Computer Programming 11],7,1,YES,William,0,NO
3,Arts Education / ADST [Culinary Arts 11],10,1,YES,Dana,0,NO
4,Arts Education / ADST [Culinary Arts 11],10,2,YES,Dana,0,NO


In [4]:
# Partition the offered courses into ShortCourses (ABC) and LongCourses (DEFGHI)
# We only consider the CourseID, not repeated sections of the course
ShortCourses=[]
LongCourses=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][2]==1:
        if CourseInfo[m][3]=="YES":
            ShortCourses.append(CourseID)
        else:
            LongCourses.append(CourseID)
            
# Partition the offered courses into OneSection and TwoSection
AllCourses=[CourseInfo[m][1] for m in range(len(CourseInfo))]
OneSection=[]
TwoSections=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][2]==2:
        TwoSections.append(CourseID)   
for CourseID in AllCourses:
    if not CourseID in TwoSections:
        OneSection.append(CourseID)
        
# Identify all Core Courses
CoreCourses=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][6]=="YES" and CourseInfo[m][2]==1:
        CoreCourses.append(CourseID)           
        
# Determine who is teaching each course
TeacherOfCourse = [0 for j in range(100)]
for j in range(len(CourseInfo)):
    CourseID = CourseInfo[j][1]
    TeacherName = CourseInfo[j][4]
    TeacherOfCourse[CourseID] = TeacherName

In [5]:
# Determine the preference coefficients based on the student data
# We have 10 points for a Core Course, 3 points for a Grade 12 elective
# and 1 point for a Grade 11 elective.

n = len(StudentInfo)
m = 100
P = np.zeros((n,m), dtype=int)
for i in range(n):
    for j in range(m):
        P[i,j]=-1

MaxScore=0
for i in range(n):
    for j in range(2,12):
        StudentGrade = StudentInfo[i][1]
        CoursePick = StudentInfo[i][j]
        if CoursePick > 0:
            if CoursePick in CoreCourses: P[i,CoursePick]=10
            else:
                if StudentGrade==12: P[i,CoursePick]=3
                if StudentGrade==11: P[i,CoursePick]=1

In [6]:
# Create Conflict Graph.  First start with the ShortCourses (ABC)

Vertices = list(set(ShortCourses).intersection(OneSection))
Vertices.sort()
v = len(Vertices)
M = np.zeros((v,v), dtype=int)
for x in range(v):
    for y in range(x+1,v):
        for i in range(n):
            if P[i,Vertices[x]]>0 and P[i,Vertices[y]]>0 :
                M[x,y] += min(P[i,Vertices[x]], P[i,Vertices[y]]) 
                

# Add 100 points for each pair of courses x and y taught by the same teacher

for x in range(v):
    for y in range(x+1,v):
        if TeacherOfCourse[Vertices[x]] == TeacherOfCourse[Vertices[y]]:
            M[x,y] += 100
            

# Find the chromatic number of the Short Conflict Graph.  If this number is above 3,
# eliminate the edges with weight 1.

S0=nx.Graph()
S0.add_nodes_from(Vertices)
for j1 in range(v):
    for j2 in range(j1+1, v):
        if M[j1,j2] > 0:
            S0.add_edge(Vertices[j1], Vertices[j2])
print("Chromatic number of S0 is", gp.chromatic_number(S0))

S1=nx.Graph()
S1.add_nodes_from(Vertices)
for j1 in range(v):
    for j2 in range(j1+1, v):
        if M[j1,j2] > 1:
            S1.add_edge(Vertices[j1], Vertices[j2])
print("Chromatic number of S1 is", gp.chromatic_number(S1))


# Create Conflict Graph.  Now do the long courses (DEFGHI)

LVertices = list(set(LongCourses).intersection(OneSection))
LVertices.sort()
v = len(LVertices)
M = np.zeros((v,v), dtype=int)
for x in range(v):
    for y in range(x+1,v):
        for i in range(n):
            if P[i,LVertices[x]]>0 and P[i,LVertices[y]]>0 :
                M[x,y] += min(P[i,LVertices[x]],P[i,LVertices[y]]) 
                
# Add 100 points for each pair of courses x and y taught by the same teacher

for x in range(v):
    for y in range(x+1,v):
        if TeacherOfCourse[LVertices[x]] == TeacherOfCourse[LVertices[y]]:
            M[x,y] += 100
            

# Find the chromatic number of the Conflict Graph.  If this number is above 3,
# eliminate the edges with weight 1.

L0=nx.Graph()
L0.add_nodes_from(LVertices)
for j1 in range(v):
    for j2 in range(j1+1, v):
        if M[j1,j2] > 0:
            L0.add_edge(LVertices[j1], LVertices[j2])
print("Chromatic number of L0 is", gp.chromatic_number(L0))

L1=nx.Graph()
L1.add_nodes_from(LVertices)
for j1 in range(v):
    for j2 in range(j1+1, v):
        if M[j1,j2] > 1:
            L1.add_edge(LVertices[j1], LVertices[j2])
print("Chromatic number of L1 is", gp.chromatic_number(L1))

Chromatic number of S0 is 5
Chromatic number of S1 is 3
Chromatic number of L0 is 7
Chromatic number of L1 is 6


In [7]:
FinalStats = [[0,0] for val in range(100)]

for trial in range(100):
    
    start_time = time.time()
    
    flag=0
    while flag==0:
        Colouring = nx.greedy_color(S1, strategy = 'random_sequential', interchange=True)
        Bundle1 = []
        Bundle2 = []
        Bundle3 = []
        for v in Vertices:
            if Colouring.get(v)==0: Bundle1.append(v)
            if Colouring.get(v)==1: Bundle2.append(v)
            if Colouring.get(v)==2: Bundle3.append(v)
        if len(Bundle1 + Bundle2 + Bundle3) == 14:
            flag=1
            
    solver = pywraplp.Solver('St. Margarets School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    AllStudents = range(n)

    Sections = [1,2]
    ShortBlocks = [1,2,3]

    # define boolean variables
    x = {}
    for s in Sections:
        for j in ShortCourses:
            for k in ShortBlocks:
                x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

    y = {}
    for i in AllStudents:
        for j in ShortCourses:
            for k in ShortBlocks:
                y[i,j,k] = solver.IntVar(0,1, 'y[%d,%d,%d]' % (i,j,k)) 


    # CONSTRAINT 1: For two-section courses, ensure each section is offered once
    for j in set(ShortCourses).intersection(TwoSections):
        solver.Add(sum(x[1,j,k] for k in ShortBlocks) == 1)
        solver.Add(sum(x[2,j,k] for k in ShortBlocks) == 1)

    # CONSTRAINT 2: For single-section courses, ensure x[2,j,k] is set to 0
    for j in set(ShortCourses).intersection(OneSection):
        solver.Add(sum(x[1,j,k] for k in ShortBlocks) == 1)
        solver.Add(sum(x[2,j,k] for k in ShortBlocks) == 0)

    # CONSTRAINT 3: Two sections of the same course can't be offered in the same block
    for j in ShortCourses:
        for k in ShortBlocks:
            solver.Add(x[1,j,k] + x[2,j,k] <= 1)

    # CONSTRAINT 4: At most eight classes per block
    for k in ShortBlocks:
        solver.Add(sum(x[1,j,k] + x[2,j,k] for j in ShortCourses) <= 8)
        
    # CONSTRAINT 5: No teacher can teach two classes in the same block
    TeacherList = set([CourseInfo[z][4] for z in range(len(CourseInfo))])
    for Teacher in TeacherList:
        ShortCourseList=[]
        for z in range(len(CourseInfo)):
            if CourseInfo[z][4]==Teacher and CourseInfo[z][1] in ShortCourses:
                ShortCourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
        if len(ShortCourseList)>1:
            for k in ShortBlocks:
                solver.Add(sum(x[ShortCourseList[j][0],ShortCourseList[j][1],k] 
                               for j in range(len(ShortCourseList))) <= 1)

    # CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
    for z in range(len(CourseInfo)):
        if CourseInfo[z][5] in ShortBlocks:
            s = CourseInfo[z][2]
            j = CourseInfo[z][1]
            k = CourseInfo[z][5]
            solver.Add(x[s,j,k] == 1)

    # CONSTRAINT 7: Each student takes at most one course per block
    for k in ShortBlocks: 
        for i in AllStudents:
            solver.Add(sum(y[i,j,k] for j in ShortCourses) <= 1)

    # CONSTRAINT 8: NO student can take the same course twice       
    for j in ShortCourses:
        for i in AllStudents:
            solver.Add(sum(y[i,j,k] for k in ShortBlocks) <= 1)  

    # CONSTRAINT 9: No student can take a course in a block when that course isn't offered
    for i in AllStudents:
        for j in ShortCourses:
            for k in ShortBlocks:
                solver.Add(y[i,j,k] <= x[1,j,k]+x[2,j,k])

    # CONSTRAINT 10: Every two-section course must have at most 18 students
    for j in set(ShortCourses).intersection(TwoSections):
        for k in ShortBlocks:
                solver.Add(sum(y[i,j,k] for i in AllStudents) <= 18)

    # CONSTRAINT 11: Do not assign course j to a student i if P[i,j]<0
    for i in AllStudents:
        for j in ShortCourses:
            if P[i,j]<0:
                for k in ShortBlocks:
                    solver.Add(y[i,j,k]==0)


    # CONSTRAINT 12: Use graph theory to pre-bundle some one-section courses
    for MySet in [Bundle1, Bundle2, Bundle3]:   
        for v1 in range(len(MySet)):
            for v2 in range(v1+1, len(MySet)):
                for k in ShortBlocks:
                    solver.Add(x[1,MySet[v1],k]==x[1,MySet[v2],k])


    solver.Maximize(solver.Sum(P[i,j]*y[i,j,k] for i in AllStudents for j in ShortCourses for k in ShortBlocks))
    sol = solver.Solve()
    
    ShortTotal = round(solver.Objective().Value())   
    
    flag=0
    while flag==0:
        Colouring = nx.greedy_color(L1, strategy = 'random_sequential', interchange=True)
        Bundle4 = []
        Bundle5 = []
        Bundle6 = []
        Bundle7 = []
        Bundle8 = []
        Bundle9 = []
        for v in LVertices:
            if Colouring.get(v)==0: Bundle4.append(v)
            if Colouring.get(v)==1: Bundle5.append(v)
            if Colouring.get(v)==2: Bundle6.append(v)
            if Colouring.get(v)==3: Bundle7.append(v)
            if Colouring.get(v)==4: Bundle8.append(v)
            if Colouring.get(v)==5: Bundle9.append(v)

        if len(Bundle4 + Bundle5 + Bundle6 + Bundle7 + Bundle8 + Bundle9) == 15:
            flag=1

        
    # Optimize long blocks

    solver = pywraplp.Solver('St. Margarets School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    AllStudents = range(n)

    Sections = [1,2]
    LongBlocks = [4,5,6,7,8,9]

    # define boolean variables
    x = {}
    for s in Sections:
        for j in LongCourses:
            for k in LongBlocks:
                x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

    y = {}
    for i in AllStudents:
        for j in LongCourses:
            for k in LongBlocks:
                y[i,j,k] = solver.IntVar(0,1, 'y[%d,%d,%d]' % (i,j,k)) 


    # CONSTRAINT 1: For two-section courses, ensure each section is offered once
    for j in set(LongCourses).intersection(TwoSections):
        solver.Add(sum(x[1,j,k] for k in LongBlocks) == 1)
        solver.Add(sum(x[2,j,k] for k in LongBlocks) == 1)

    # CONSTRAINT 2: For single-section courses, ensure x[2,j,k] is set to 0
    for j in set(LongCourses).intersection(OneSection):
        solver.Add(sum(x[1,j,k] for k in LongBlocks) == 1)
        solver.Add(sum(x[2,j,k] for k in LongBlocks) == 0)

    # CONSTRAINT 3: Two sections of the same course can't be offered in the same block
    for j in LongCourses:
        for k in LongBlocks:
            solver.Add(x[1,j,k] + x[2,j,k] <= 1)

    # CONSTRAINT 4: At most five classes per block
    for k in LongBlocks:
        solver.Add(sum(x[1,j,k] + x[2,j,k] for j in LongCourses) <= 5)

    # CONSTRAINT 5: No teacher can teach two classes in the same block
    TeacherList = set([CourseInfo[z][4] for z in range(len(CourseInfo))])
    for Teacher in TeacherList:
        LongCourseList=[]
        for z in range(len(CourseInfo)):
            if CourseInfo[z][4]==Teacher and CourseInfo[z][1] in LongCourses:
                LongCourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
        if len(LongCourseList)>1:
            for k in LongBlocks:
                solver.Add(sum(x[LongCourseList[j][0],LongCourseList[j][1],k] 
                               for j in range(len(LongCourseList))) <= 1)

    # CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
    for z in range(len(CourseInfo)):
        if CourseInfo[z][5] in LongBlocks:
            s = CourseInfo[z][2]
            j = CourseInfo[z][1]
            k = CourseInfo[z][5]
            solver.Add(x[s,j,k] == 1)

    # CONSTRAINT 7: Each student takes at most one course per block
    for k in LongBlocks: 
        for i in AllStudents:
            solver.Add(sum(y[i,j,k] for j in LongCourses) <= 1)

    # CONSTRAINT 8: NO student can take the same course twice       
    for j in LongCourses:
        for i in AllStudents:
            solver.Add(sum(y[i,j,k] for k in LongBlocks) <= 1)  

    # CONSTRAINT 9: No student can take a course in a block when that course isn't offered
    for i in AllStudents:
        for j in LongCourses:
            for k in LongBlocks:
                solver.Add(y[i,j,k] <= x[1,j,k]+x[2,j,k])

    # CONSTRAINT 10: Every two-section course must have at most 18 students
    for j in set(LongCourses).intersection(TwoSections):
        for k in LongBlocks:
                solver.Add(sum(y[i,j,k] for i in AllStudents) <= 18)


    # CONSTRAINT 11: Do not assign course j to a student i if P[i,j]<0
    for i in AllStudents:
        for j in LongCourses:
            if P[i,j]<0:
                for k in LongBlocks:
                    solver.Add(y[i,j,k]==0)


    # CONSTRAINT 12: Use graph theory to pre-bundle some one-section courses
    for MySet in [Bundle4, Bundle5, Bundle6, Bundle7, Bundle8, Bundle9]:   
        for v1 in range(len(MySet)):
            for v2 in range(v1+1, len(MySet)):
                for k in LongBlocks:
                    solver.Add(x[1,MySet[v1],k]==x[1,MySet[v2],k])


    solver.Maximize(solver.Sum(P[i,j]*y[i,j,k] for i in AllStudents for j in LongCourses for k in LongBlocks))
    sol = solver.Solve()
    
    LongTotal = round(solver.Objective().Value())   
    
    # compute runtime
    TotalTime = time.time() - start_time
    
    print(trial, ShortTotal + LongTotal, round(TotalTime, 1))
    
    FinalStats[trial][0] = ShortTotal + LongTotal
    FinalStats[trial][1] = round(TotalTime, 1)


0 2159 6.2
1 2147 6.2
2 2149 3.2
3 2149 4.8
4 2152 5.1
5 2158 4.5
6 2148 3.8
7 2156 3.5
8 2151 3.6
9 2157 5.9
10 2157 4.4
11 2158 2.0
12 2158 4.9
13 2154 3.3
14 2153 4.7
15 2147 4.7
16 2151 3.8
17 2149 4.0
18 2154 2.1
19 2148 5.3
20 2151 3.1
21 2150 3.9
22 2157 4.1
23 2145 3.9
24 2164 4.0
25 2147 4.9
26 2149 3.0
27 2154 2.0
28 2150 3.3
29 2149 4.2
30 2149 3.3
31 2156 2.0
32 2158 5.8
33 2145 3.1
34 2153 4.9
35 2166 5.9
36 2145 4.5
37 2147 3.6
38 2156 5.7
39 2141 4.6
40 2163 8.3
41 2156 4.6
42 2149 3.8
43 2167 2.8
44 2159 3.4
45 2145 3.9
46 2145 4.2
47 2158 5.6
48 2161 3.7
49 2158 5.0
50 2153 6.8
51 2149 3.3
52 2164 2.5
53 2155 4.9
54 2147 3.5
55 2153 2.1
56 2157 2.2
57 2154 3.4
58 2149 6.7
59 2150 5.3
60 2159 4.6
61 2145 3.1
62 2145 3.5
63 2156 1.7
64 2140 6.5
65 2157 4.4
66 2154 4.4
67 2145 5.1
68 2149 4.0
69 2164 4.8
70 2149 4.3
71 2158 4.9
72 2151 5.4
73 2148 3.8
74 2152 3.3
75 2149 3.1
76 2147 4.7
77 2147 6.0
78 2152 3.4
79 2149 3.9
80 2145 3.7
81 2166 3.9
82 2164 2.1
83 2157 4.0
84

In [13]:
sum(FinalStats[i][0] for i in range(100))/100

2153.36

In [14]:
min(FinalStats[i][0] for i in range(100)),max(FinalStats[i][0] for i in range(100))

(2140, 2170)

In [15]:
sum(FinalStats[i][1] for i in range(100))/100

4.147

In [17]:
min(FinalStats[i][1] for i in range(100)),max(FinalStats[i][1] for i in range(100))

(1.7, 8.3)